In [115]:
import autograd.numpy as np
import matplotlib.pyplot as plt
from autograd import grad, elementwise_grad, hessian, jacobian

import sys
sys.path.insert(0, "../../project_2/src")
from SGD import minibatch

In [116]:
# Inspired by mortens example code in week 43

def activation_function(z):
    return np.maximum(0, z)

def activation_out(z):
    return z

"""
network_shape = [inputs, w1, w2, ..., wL, outputs]

W = wwwwb
    wwwwb
"""
def initialize_params(network_shape):
    P = []
    for i in range(1, len(network_shape)):
        k = network_shape[i-1]
        j = network_shape[i]
        P.append(np.random.randn(j, k) * np.sqrt(2) / np.sqrt(k))
        P[i-1] = np.concatenate((P[i-1], np.zeros(j).reshape(-1,1)), axis=1)

    return P

def Network(t, P):
    # Assumes the input t to be a scalar, returns a 2d-row vector e.g. shape=[1,6]
    a = t.reshape(-1,1)
    for P_i in P:
        #a = np.concatenate((a, np.ones(np.size(a, 0)).reshape(-1, 1)), axis=1) 
        a = np.concatenate((a, np.ones((1,1))), axis=1) 
        z = np.matmul(a, np.transpose(P_i))
        a = activation_function(z)
    
    return activation_out(z) 

def Network_predict(t, P):
    # For predictions
    # Assumes the input t to be a 1d-array, 
    # returns a matrix where each row corresponds to a vector for a particular t
    a = t.reshape(-1,1)
    for P_i in P:
        a = np.concatenate((a, np.ones(np.size(a, 0)).reshape(-1, 1)), axis=1) 
        #a = np.concatenate((a, np.ones((1,1))), axis=1) 
        z = np.matmul(a, np.transpose(P_i))
        a = activation_function(z)
    
    return activation_out(z) 

def optimize(t, P, A, x_0, N_minibatches, learning_rate, n_epochs):
    # Assumes t is a 1d-array.
    assert N_minibatches <= np.size(t, 0)

    cost_func_grad = grad(costfunction, 1) # Check which grad-call is correct.
    
    for epoch in range(n_epochs):
        mb = minibatch(t, N_minibatches)
        for i in range(N_minibatches):
            t_mb = t[mb[i]]
            M = np.size(t_mb, 0)
            # compute gradients of weights
            cost_grad = cost_func_grad(t_mb, P, A, x_0)
            for l in range(len(P)):
                P[l] -= learning_rate * cost_grad[l]
    return P

def g_trial(t, P, x_0):
    #assumes the input t to be a scalar, x_0 is 1d-row-vector which broadcasts to e.g. (1,6)
    
    return np.exp(-t*100)*x_0 + (1-np.exp(-t*100))*Network(t,P)

def g_trial_predict(t, P, x_0):
    # For predictions
    #assumes the input t to be a 1d-array, broadcasts along the rows of Network_predict's output
    t = t.reshape(-1,1)
    return np.exp(-t*100)*x_0 + (1-np.exp(-t*100))*Network_predict(t,P)

def costfunction(t, P, A, x_0):
    
    cost = 0
    g_grad = jacobian(g_trial,0) # Check that this is the correct grad-call
    
    for time in t:
        d_dt = g_grad(time,P,x_0).reshape(-1,1) # should have shape (eigenvector_length,1)
        x_t = g_trial(time,P,x_0).reshape(-1,1) #check shape, should be (eigenvector_length,1)
        # Right hand side is: -x + [x.T @ x * A + (1 - x.T @ A @ x) @ I] @ x where x is x_t is a column vector
        right_side = (-x_t + (x_t.T @ x_t * A + (1 - x_t.T @ A @ x_t) * np.identity(np.size(x_t,0))) @ x_t)
#        print(x_t)
#        print(right_side)
#         right_side = -x_t + np.matmul( # -x + [
#                                 np.matmul(np.transpose(x_t), x_t) * A #(x.T @ x) * A
#                                 + ( 1 - np.matmul(np.transpose(x_t),np.matmul(A, x_t))) # + (1-x.t @ A @ x)
#                                 * np.identity(np.size(x_t, 0)) # * I
#                                 , x_t) # ] @ x
        cost = cost + np.sum((d_dt - right_side)**2) / np.size(x_t,0) 
                                                              
        

    return cost / np.size(t)

In [141]:
t = np.linspace(0,1,100)
Q = np.random.randn(6,6)
A = (np.transpose(Q) + Q) /2 #standard trick for making symmetric real
A = np.array([[0.7663, 0.4283, -0.3237, -0.4298, -0.1438], [0.4283,0.2862,0.0118,-0.2802,0.1230]
              , [-0.3237,0.0118,-0.9093,-0.4383,0.7684],[-0.4298,-0.2802,-0.4384,-0.0386,-0.1315]
              , [-0.1438,0.1230,0.7684,-0.1315,-0.4480]]) #matrix from Yi et al.

x_0 = np.random.randn(np.size(A,0)) #initial guess for eigenvector.

network_shape = [1, 10, np.size(x_0)] #output must match eigenvector length
P = initialize_params(network_shape)

# Solution from np.linalg.eigh

eigval, eigvec = np.linalg.eigh(A)
print(eigval)
print(eigvec)

[-1.56887992 -0.48134608 -0.03749509  0.51368892  1.23063218]
[[-0.11591938  0.36630714 -0.31514082 -0.31481908  0.80867607]
 [-0.03697112 -0.06476226  0.82605531  0.3081914   0.46592883]
 [-0.81391888  0.16545258 -0.18429373  0.52208713 -0.06018593]
 [-0.22777979  0.66454556  0.41248819 -0.46070518 -0.35227801]
 [ 0.52043083  0.6266016  -0.1191982   0.56664166 -0.0350885 ]]


In [196]:
P = optimize(t,P,A,x_0,10,0.0005,10)
x_test = g_trial_predict(t,P,x_0)
x_test = x_test/np.linalg.norm(x_test,axis=1)[:,np.newaxis]

In [197]:
print(x_test[0])
print(x_test[-1])
print(eigvec[:,-1])
print(np.sum(np.abs(np.abs(x_test[-1])-np.abs(eigvec[:,-1]))))

[ 0.8191699   0.43716885 -0.1283537  -0.34822712 -0.01035701]
[ 0.80748771  0.47444741 -0.05946983 -0.34356713 -0.03589184]
[ 0.80867607  0.46592883 -0.06018593 -0.35227801 -0.0350885 ]
0.019937264578885575


In [30]:
#Forward Euler for Yi et al diff.eq

x_t = x_0.reshape(-1,1)
dt = 0.01
print(x_t) # initial guess
for i in range(500):
    x_t = x_t + (-x_t + (x_t.T @ x_t * A + (1 - x_t.T @ A @ x_t) * np.identity(5)) @ x_t)*dt
print(x_t/np.sqrt(x_t.T @ x_t)) # normalized eigenvector
print(x_t.T @ A @ x_t / (x_t.T @ x_t)) # eigenvalue

[[-1.42335238]
 [ 0.45299681]
 [ 0.01039818]
 [ 2.37626212]
 [-2.0335502 ]]
[[-0.8086801 ]
 [-0.46592499]
 [ 0.06020916]
 [ 0.35226867]
 [ 0.03510045]]
[[1.2306343]]
